# Samuel Thelin, samthe-1 & Albin Mårtensson, albmrt-1

In [73]:
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.datasets import load_digits
from sklearn.datasets import load_breast_cancer


In [74]:
def kNN_predict(testSet, trainingSet, trainingLabel, k_nearest):
    numerable = testSet.shape[0]
    Lpred = np.zeros(numerable, dtype=trainingLabel.dtype)
    
    # cast as int to not get bugs like before
    testSet = testSet.astype(int)
    trainingSet = trainingSet.astype(int)

    for i in range(numerable):
        distances = np.sqrt(np.sum((trainingSet - testSet[i, :]) ** 2, axis=1)) # L2
        nearest_indices = np.argsort(distances)[:k_nearest]
        nearest_labels = trainingLabel[nearest_indices]
        label_counts = np.bincount(nearest_labels, minlength=10)
        
        # break ties if needed.
        if np.count_nonzero(label_counts == label_counts.max()) > 1:
            winner = trainingLabel[nearest_indices[0]]
        else:
            winner = np.argmax(label_counts)
        Lpred[i] = winner
    
    return Lpred

In [75]:
def find_optimal_k(training_set, training_labels, k_values, num_folds=3):
    best_k = None
    best_accuracy = 0
    accuracies_for_k = {k: [] for k in k_values}
    
    kf = KFold(n_splits=num_folds, shuffle=True)
    
    for k in k_values:
        fold_accuracies = []
        
        for train_index, val_index in kf.split(training_set):
            X_train_fold, X_val_fold = training_set[train_index], training_set[val_index]
            y_train_fold, y_val_fold = training_labels[train_index], training_labels[val_index]
            
            predictions = kNN_predict(X_val_fold, X_train_fold, y_train_fold, k)
            
            accuracy = np.mean(predictions == y_val_fold)
            fold_accuracies.append(accuracy)
        
        avg_accuracy = np.mean(fold_accuracies)
        accuracies_for_k[k].append(avg_accuracy)
        
        if avg_accuracy > best_accuracy:
            best_accuracy = avg_accuracy
            best_k = k
    
    return best_k, best_accuracy, accuracies_for_k

In [76]:
def test_datasets(dataset, dataset_label):
    X = dataset.data
    y = dataset.target

    X = X.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    k_values = list(range(1, 11)) 

    best_k, best_accuracy, accuracies_for_k = find_optimal_k(X_train, y_train, k_values, num_folds=5)
    print('\nDataset Label: ',dataset_label,'\n')
    print(f"\noptimal k: {best_k} with cross-validated accuracy: {best_accuracy:.4f}\n")
    for k, accuracy in accuracies_for_k.items():
        print(f"With k = {k}: {accuracy[0]:.4f}")
    #print("accuracies for each k:", accuracies_for_k,'\n')

    y_pred = kNN_predict(X_test, X_train, y_train, best_k)
    test_accuracy = accuracy_score(y_test, y_pred)

    print(f"\ntest set accuracy with k={best_k}: {test_accuracy:.4f}\n===================================================")

In [ ]:
test_datasets(load_iris(),'Iris')
test_datasets(load_wine(),'Wine')
test_datasets(load_breast_cancer(),'Breast Cancer')
test_datasets(load_digits(),'Digits')

# MNIST test

from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score

(X_train, y_train), (X_test, y_test) = mnist.load_data()

num_sample = 500  
Tr_set = X_train[:num_sample].reshape(num_sample, -1).astype(int)
Ltr_set = y_train[:num_sample]

X_test_flat = X_test[:num_sample].reshape(num_sample, -1).astype(int)
y_test_subset = y_test[:num_sample]

k_values = list(range(1, 11))

best_k, best_accuracy, accuracies_for_k = find_optimal_k(Tr_set, Ltr_set, k_values)
print('\nDataset Label: MNIST\n')
print(f"\nOptimal k: {best_k} with cross-validated accuracy: {best_accuracy:.4f}\n")
for k, accuracy in accuracies_for_k.items():
    print(f"With k = {k}: {accuracy[0]:.4f}")

y_pred = kNN_predict(X_test_flat, Tr_set, Ltr_set, best_k)
test_accuracy = accuracy_score(y_test_subset, y_pred)

print(f"\nTest set accuracy with k={best_k}: {test_accuracy:.4f}")
print("=" * 50)



Dataset Label:  Iris 


optimal k: 4 with cross-validated accuracy: 0.9500

With k = 1: 0.8750
With k = 2: 0.9333
With k = 3: 0.9083
With k = 4: 0.9500
With k = 5: 0.9417
With k = 6: 0.9417
With k = 7: 0.9333
With k = 8: 0.9500
With k = 9: 0.9250
With k = 10: 0.9500

test set accuracy with k=4: 0.9000

Dataset Label:  Wine 


optimal k: 6 with cross-validated accuracy: 0.7325

With k = 1: 0.6897
With k = 2: 0.7180
With k = 3: 0.6847
With k = 4: 0.6980
With k = 5: 0.7047
With k = 6: 0.7325
With k = 7: 0.7042
With k = 8: 0.6830
With k = 9: 0.7057
With k = 10: 0.6133

test set accuracy with k=6: 0.8056

Dataset Label:  Breast Cancer 


optimal k: 9 with cross-validated accuracy: 0.9407

With k = 1: 0.9209
With k = 2: 0.9253
With k = 3: 0.9319
With k = 4: 0.9319
With k = 5: 0.9319
With k = 6: 0.9253
With k = 7: 0.9253
With k = 8: 0.9253
With k = 9: 0.9407
With k = 10: 0.9297

test set accuracy with k=9: 0.9211

Dataset Label:  Digits 


optimal k: 2 with cross-validated accuracy: 0.9910



In [79]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

class EchoStateNetwork:
    def __init__(self, input_dim, reservoir_dim, output_dim, spectral_radius=0.99, input_scaling=0.25, reg_param=1e-8):
        self.input_dim = input_dim
        self.reservoir_dim = reservoir_dim
        self.output_dim = output_dim
        self.spectral_radius = spectral_radius
        self.input_scaling = input_scaling
        self.reg_param = reg_param

        self.W_in = np.random.uniform(-1, 1, (reservoir_dim, input_dim)) * input_scaling
        self.W = np.random.uniform(-1, 1, (reservoir_dim, reservoir_dim))
        max_eigenvalue = max(abs(np.linalg.eigvals(self.W)))
        self.W *= spectral_radius / max_eigenvalue
        self.W_out = None

    def _update_reservoir(self, x_t, r_prev):
        return np.tanh(np.dot(self.W_in, x_t) + np.dot(self.W, r_prev))

    def train(self, X_train, y_train_onehot):
        reservoir_states = []
        for signal in X_train:
            r_prev = np.zeros(self.reservoir_dim)
            r_prev = self._update_reservoir(signal, r_prev)
            reservoir_states.append(r_prev)
        reservoir_states = np.array(reservoir_states)

        extended_states = np.hstack([reservoir_states, np.ones((reservoir_states.shape[0], 1))])
        self.W_out = np.linalg.solve(
            np.dot(extended_states.T, extended_states) + self.reg_param * np.eye(extended_states.shape[1]),
            np.dot(extended_states.T, y_train_onehot)
        )

    def predict(self, X_test):
        predictions = []
        for signal in X_test:
            r_prev = np.zeros(self.reservoir_dim)
            r_prev = self._update_reservoir(signal, r_prev)
            extended_state = np.hstack([r_prev, 1])
            y_pred = np.dot(self.W_out.T, extended_state)
            predictions.append(y_pred)
        return np.array(predictions)


iris = load_iris()
X = iris.data 
y = iris.target

encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y.reshape(-1, 1))

X_train, X_test, y_train_onehot, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

X_train_flat = X_train.astype(np.float32)
X_test_flat = X_test.astype(np.float32)

input_dim = X_train_flat.shape[1]  
reservoir_dim = 1000
output_dim = y_train_onehot.shape[1] 

esn = EchoStateNetwork(input_dim, reservoir_dim, output_dim)
esn.train(X_train_flat, y_train_onehot)

y_pred_logits = esn.predict(X_test_flat)
y_pred = np.argmax(y_pred_logits, axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred)
print(f"Test set accuracy: {accuracy:.4f}")


Test set accuracy: 0.9667


In [ ]:
def test_ESN_datasets(dataset, name):
    X = dataset['data']
    y = dataset['target']

    encoder = OneHotEncoder(sparse_output=False)
    y_onehot = encoder.fit_transform(y.reshape(-1, 1))

    X_train, X_test, y_train_onehot, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

    X_train_flat = X_train.astype(np.float32)
    X_test_flat = X_test.astype(np.float32)

    input_dim = X_train_flat.shape[1]  
    reservoir_dim = 1000
    output_dim = y_train_onehot.shape[1] 

    esn = EchoStateNetwork(input_dim, reservoir_dim, output_dim)
    esn.train(X_train_flat, y_train_onehot)

    y_pred_logits = esn.predict(X_test_flat)
    y_pred = np.argmax(y_pred_logits, axis=1)
    y_true = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    print('Dataset Label: ',name)
    print(f"Test set accuracy: {accuracy:.4f}")
    print('\n===================================================')

def fit_MNIST_for_test_method():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # Combine training and test data
    X = np.vstack((X_train, X_test))  # Shape: (70000, 28, 28)
    y = np.hstack((y_train, y_test))  # Shape: (70000,)

    # Flatten images
    X_flat = X.reshape(X.shape[0], -1)  # Shape: (70000, 784)

    # Return a dictionary to match `test_ESN_datasets` expectations
    return {'data': X_flat, 'target': y}


In [ ]:
test_ESN_datasets(load_iris(),'Iris')
test_ESN_datasets(load_wine(),'Wine')
test_ESN_datasets(load_breast_cancer(),'Breast Cancer')
test_ESN_datasets(load_digits(),'Digits')
mnist_dataset = fit_MNIST_for_test_method()
test_ESN_datasets(mnist_dataset, "MNIST")


Dataset Label:  Iris
Test set accuracy: 0.9667

Dataset Label:  Wine
Test set accuracy: 0.8333

Dataset Label:  Breast Cancer
Test set accuracy: 0.8596

Dataset Label:  Digits
Test set accuracy: 0.9361

Dataset Label:  MNIST
Test set accuracy: 0.9150

